In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

In [2]:
df= pd.read_csv('dataset_train+test2 copy.csv', sep=",")

In [3]:
df.head()

,date,Temperature,Humidity,Light,CO2,Occupancy,Ora,day,hour,absH,weekend,out_of_turn
0,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,1,2,4,17:51:00,17,0.0,1.0
1,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,1,2,4,17:51:59,17,0.0,1.0
2,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,1,2,4,17:53:00,17,0.0,1.0
3,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,1,2,4,17:54:00,17,0.0,1.0
4,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,1,2,4,17:55:00,17,0.0,1.0


In [4]:
del df['date']

In [5]:
df = df[['Temperature', 'Light', 'CO2', 'Occupancy', ]]

In [6]:
df.head()

,Temperature,Light,CO2,Occupancy
0,23.18,426.0,721.25,1
1,23.15,429.5,714.00,1
2,23.15,426.0,713.50,1
3,23.15,426.0,708.25,1
4,23.10,426.0,704.50,1


In [7]:
from sklearn.model_selection import train_test_split, cross_val_score 

In [8]:
y_name= "Occupancy"
attributes = [col for col in df.columns if col != y_name]
X = df[attributes].values
y = df[y_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [9]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
lasso = Lasso()
print(cross_val_score(lasso, X, y, cv=3))


[0.78138782 0.85350968 0.82374437]


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# DECISON TREE

In [11]:
X_train.shape, X_test.shape

((12526, 3), (5369, 3))

In [12]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=None, 
                             min_samples_split=2, min_samples_leaf=1)
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [13]:
for col, imp in zip(attributes, clf.feature_importances_):
    print(col, imp)

Temperature 0.014866639754662524
Light 0.9581132842378615
CO2 0.02702007600747615


In [14]:
import pydotplus
from sklearn import tree
from IPython.display import Image


# CROSS VALIDATION

In [15]:
from sklearn.model_selection import cross_val_score

In [16]:
scores = cross_val_score(clf, X, y, cv=10)
print('Accuracy: %0.4f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

scores = cross_val_score(clf, X, y, cv=10, scoring='f1_macro')
print('F1-score: %0.4f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

Accuracy: 0.9633 (+/- 0.06)
F1-score: 0.9388 (+/- 0.12)


In [17]:
scores

array([0.94519132, 0.96953367, 0.83597195, 0.9768407 , 0.99497914,
       0.90419202, 0.95076422, 0.83399924, 0.98671119, 0.9900144 ])

# GRID SEARCH E STIMA DEI PARAMETRI

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [19]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [20]:
param_list = {'min_samples_split': [2, 5, 10, 20],
              'min_samples_leaf': [1, 5, 10, 20],
             }

grid_search = GridSearchCV(clf, param_grid=param_list)
grid_search.fit(X, y)
clf = grid_search.best_estimator_

In [21]:
param_list = {'max_depth': [None] + list(np.arange(2, 20)),
              'min_samples_split': [2, 5, 10, 20, 30, 50, 100],
              'min_samples_leaf': [1, 5, 10, 20, 30, 50, 100],
             }

random_search = RandomizedSearchCV(clf, param_distributions=param_list, 
                                   n_iter=100)
random_search.fit(X, y)
clf = random_search.best_estimator_

In [22]:
report(random_search.cv_results_, n_top=3)

Model with rank: 1
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 100, 'min_samples_leaf': 10, 'max_depth': 3}

Model with rank: 2
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 100, 'max_depth': 13}

Model with rank: 2
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 20, 'min_samples_leaf': 100, 'max_depth': 6}

Model with rank: 2
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 100, 'min_samples_leaf': 100, 'max_depth': 16}

Model with rank: 2
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 100, 'max_depth': 2}

Model with rank: 2
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 100, 'max_depth': 14}

Model with rank: 2
Mean validation score: 0.990 (std: 0.008)
Parameters: {'min_samples_split': 100, 'min_samples_leaf': 100, 'max_depth': None}

Model wi